In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

model = Ollama(model='gemma2:2b', stop=["<|eot_id|>"]) # Added the stop token

def get_model_response(user_prompt, system_prompt):
    # NOTE: No f-string and no whitespace in curly braces.
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
    """
    
    prompt = PromptTemplate(
        input_variables=['system_prompt', 'user_prompt'],
        template=template
    )
    
    response = model(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

    return response

In [3]:
user_prompt = "Explain about Seoul."

system_prompt = """Think and write your step-by-step reasoning before responding.
Make sure you don't deviate from json grammar.
Please write all conversations in Korean(한국어).

You must return a value in the following format:
{"result": "text"}

You will be given human language prompts, and you need to return a JSON conformant to the ontology. Any action not in the ontology must be ignored. Here are some examples.

prompt: "What is LangChain?"
returns: {"result": "LangChain is a framework designed to help develop applications using language models."}
"""

res = get_model_response(user_prompt, system_prompt)
print(res)

/tmp/ipykernel_20639/2636955753.py:24: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = model(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))


```json
{"result": "서울은 대한민국에서 가장 큰 도시 중 하나인 곳입니다. 옛날부터 중요한 문화와 역사적 의미를 가진 도시로, 서울의 역사와 문화는 대통령의 주택, 국회, 관광 명소 등에 나타납니다. 서울은 많은 관광 명소가 있어 한국 최대 인구 중심지에 위치해 있습니다. 관광객들에게 긍정적인 경험을 제공하는 곳이라는 점에서 특별한 도시입니다."}
```


In [4]:
import re

pattern = r'^```(?:\w+)?\s*\n(.*?)(?=^```)```'
rm_codeblock_res = re.findall(pattern, res, re.DOTALL | re.MULTILINE)

In [5]:
cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', rm_codeblock_res[0].strip())

cleaned_data

'{"result": "서울은 대한민국에서 가장 큰 도시 중 하나인 곳입니다. 옛날부터 중요한 문화와 역사적 의미를 가진 도시로, 서울의 역사와 문화는 대통령의 주택, 국회, 관광 명소 등에 나타납니다. 서울은 많은 관광 명소가 있어 한국 최대 인구 중심지에 위치해 있습니다. 관광객들에게 긍정적인 경험을 제공하는 곳이라는 점에서 특별한 도시입니다."}'

In [6]:
import json

res_json = json.loads(cleaned_data)
print(res_json['result'])

서울은 대한민국에서 가장 큰 도시 중 하나인 곳입니다. 옛날부터 중요한 문화와 역사적 의미를 가진 도시로, 서울의 역사와 문화는 대통령의 주택, 국회, 관광 명소 등에 나타납니다. 서울은 많은 관광 명소가 있어 한국 최대 인구 중심지에 위치해 있습니다. 관광객들에게 긍정적인 경험을 제공하는 곳이라는 점에서 특별한 도시입니다.
